In [ ]:
import os, sys, string, time
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import xgboost
from xgboost import XGBClassifier
import joblib
import seaborn as sns
import pickle
from importlib import reload
import csv
import ROOT
from array import array

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


## Purpose

This notebook loads the BDT models trained in script 3 and saves the BDT output scores of the various samples. <br>
There is also code that merges the high BDT score bins if there are too few background events. Therefore the binning of the distributions is determined at this stage of the analysis. <br>
The notebook also contains code that applies BDT models to the adjacent mass points to the one they were trained for. This is for the purposes of a sort of "closure test" where we look at limit acquired if the adjacent BDT models is applied, just to justify the interpolation we apply between limit points. 

## Reading in files

In [ ]:
Params = {"Run":"run3", #The run number, so far either "run1" or "run3"
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":245,
          "Load_standard":True, #bkgs
          "Load_lepton_signal":False,
          "Load_lepton_dirac":False,
          "Load_pi0_signal":True,
          "Load_pi0_dirac":False,
          "Load_DetVars":False, #This is for overlay
          "Only_keep_common_DetVar_evs":True,
          "Load_Signal_DetVars":False, #Don't do here, but in seperate script
          'Load_pi0_signal_DetVars':False, #Don't do here, but in seperate script
          "Load_data":True,
          "FLATTEN":True, #Have one row per reconstructed object in the analysis dataframe
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "EXT_in_training":True,
          "dirt_in_training":True,
          "Use_logit":True,
          "nbins":5} 

loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/my_vars/"

samples = Functions.create_test_samples_list(Params)

if Params["Load_pi0_signal"] == True:
    pi0_sample_strings = [] #Unfortunately need to make, to discriminate lepton final states from pi0 final states for signal
    for pi0_point in Constants.HNL_mass_pi0_samples:
        pi0_sample_strings += [str(pi0_point)+"_pi0"]
        
if (Params["Load_lepton_signal"] == True) or (Params["Load_lepton_dirac"] == True): loc = "bdts/"
else: loc = "bdts/pi0_selection/"
        
# end_string = "_FINAL"
end_string = "_full_Finished_10"


In [ ]:
def Load_preselected_pkls(samples, Params, loc_pkls, filename, Test_filename):
    """
    For loading in preselected pkl files, including BDT test pkls. 
    """
    sample_test_dict = {}
    sig_names_list = Constants.HNL_ee_samples_names+Constants.HNL_mass_pi0_samples_names
    if Params["Load_DetVars"] == True: loc_pkls += "DetVars/"
    elif Params["Load_Signal_DetVars"] == True: loc_pkls += "Signal_DetVars/"
    elif Params['Load_pi0_signal_DetVars'] == True: loc_pkls += "Signal_DetVars/pi0/"
    
    for sample in samples:
        if (sample == "overlay") or (sample in Constants.HNL_ee_samples_names): start_str = loc_pkls + "BDT_Test_dfs/Test_"
        elif (sample == "beamoff") and (Params["EXT_in_training"] == True): start_str = loc_pkls + "BDT_Test_dfs/Test_"
        elif (sample == "dirtoverlay") and (Params["dirt_in_training"] == True): start_str = loc_pkls + "BDT_Test_dfs/Test_"
        elif sample in Constants.HNL_mass_pi0_samples_names: start_str = loc_pkls + "BDT_Test_dfs/pi0_selection/Test_"
        else: start_str = loc_pkls + "Preselected_"
        
        if (sample == "beamoff") and Params["EXT_in_training"]==True:
            sample_test_dict[sample] = pd.read_pickle(start_str+f"{sample}_"+Params["Run"]+f"_flattened{Test_filename}.pkl")
        elif (sample == "dirtoverlay") and Params["dirt_in_training"]==True:
            sample_test_dict[sample] = pd.read_pickle(start_str+f"{sample}_"+Params["Run"]+f"_flattened{Test_filename}.pkl")
        elif ((sample == "overlay") or (sample in sig_names_list)):
            sample_test_dict[sample] = pd.read_pickle(start_str+f"{sample}_"+Params["Run"]+f"_flattened{Test_filename}.pkl")
        else:
            sample_test_dict[sample] = pd.read_pickle(start_str+f"{sample}_"+Params["Run"]+f"_flattened{filename}.pkl")
    
    return sample_test_dict


In [ ]:
sample_test_dict = Load_preselected_pkls(samples, Params, loc_pkls, "_full_Finished", end_string)

In [ ]:
def Load_BDT_models(sample_dict, Params, BDT_name):
    """
    Load variable list then loads BDT models for samples.
    """
    if (Params["Load_lepton_signal"] == True) or (Params["Load_lepton_dirac"] == True): loc = "bdts/"
    else: loc = "bdts/pi0_selection/"
    
    with open(loc+f"input_vars/{BDT_name}_"+Params["Run"], "rb") as fp:   # Unpickling
        saved_variables = pickle.load(fp)
    
    bdt_vars = saved_variables
    xgb_test_dict = {}
    
    for sample in sample_dict:
        xgb_test_dict[sample] = xgboost.DMatrix(sample_dict[sample][bdt_vars])
    
    if Params["Load_lepton_signal"] == True: signal_names = Constants.HNL_ee_samples_names
    elif Params["Load_pi0_signal"] == True: signal_names = Constants.HNL_mass_pi0_samples_names
    # elif Params["Load_lepton_dirac"] == True: signal_names = Constants.HNL_ee_dirac_names
    elif Params["Load_lepton_dirac"] == True: signal_names = ["10_ee", "100_ee", "150_ee"] #Using Majorana models currently
    # elif Params["Load_pi0_dirac"] == True: signal_names = Constants.HNL_pi0_dirac_names
    elif Params["Load_pi0_dirac"] == True: signal_names = ["150_pi0", "200_pi0", "245_pi0"]
    elif Params["Load_single_file"] == True: signal_names = Params["single_file"]
    
    for HNL_mass in signal_names:
        # bdt = xgboost.Booster()
        filename = loc+Params["Run"]+f"_{HNL_mass}{BDT_name}.pkl"
        bdt = pickle.load(open(filename, "rb"))
        mass_val = int(HNL_mass.split("_")[0])
        for sample in xgb_test_dict:
            results = bdt.predict(xgb_test_dict[sample])
            sample_dict[sample][f"BDT_output_{mass_val}MeV"] = results

    return sample_dict

def OLD_Load_BDT_models(sample_dict, Params, BDT_name):
    """
    Load variable list then loads BDT models for samples, using the older .json way of saving.
    """
    if (Params["Load_lepton_signal"] == True) or (Params["Load_lepton_dirac"] == True): loc = "bdts/"
    else: loc = "bdts/pi0_selection/"
    
    with open(loc+f"input_vars/{BDT_name}_"+Params["Run"], "rb") as fp:   # Unpickling
        saved_variables = pickle.load(fp)
    
    bdt_vars = saved_variables
    xgb_test_dict = {}
    
    for sample in sample_dict:
        xgb_test_dict[sample] = xgboost.DMatrix(sample_dict[sample][bdt_vars])
    
    if Params["Load_lepton_signal"] == True: signal_names = Constants.HNL_ee_samples_names
    elif Params["Load_pi0_signal"] == True: signal_names = Constants.HNL_mass_pi0_samples_names
    # elif Params["Load_lepton_dirac"] == True: signal_names = Constants.HNL_ee_dirac_names
    elif Params["Load_lepton_dirac"] == True: signal_names = ["10_ee", "100_ee", "150_ee"] #Using Majorana models currently
    # elif Params["Load_pi0_dirac"] == True: signal_names = Constants.HNL_pi0_dirac_names
    elif Params["Load_pi0_dirac"] == True: signal_names = ["150_pi0", "200_pi0", "245_pi0"]
    elif Params["Load_single_file"] == True: signal_names = Params["single_file"]
    
    for HNL_mass in signal_names:
        bdt = xgboost.Booster()
        bdt.load_model(loc+Params["Run"]+f"_{HNL_mass}{BDT_name}.json")
        mass_val = int(HNL_mass.split("_")[0])
        for sample in xgb_test_dict:
            results = bdt.predict(xgb_test_dict[sample])
            sample_dict[sample][f"BDT_output_{mass_val}MeV"] = results

    return sample_dict


In [ ]:
sample_test_dict = Load_BDT_models(sample_test_dict, Params, end_string)

print(sample_test_dict.keys())

## Checking max BDT score (after logit transformation)

In [ ]:
reload(Constants)
if Params["Load_lepton_signal"] == True: 
    HNL_masses = Constants.HNL_mass_samples
    sample_names = Constants.HNL_ee_samples_names
    name_ends = "_ee"
if Params["Load_pi0_signal"] == True: 
    HNL_masses = Constants.HNL_mass_pi0_samples
    sample_names = Constants.HNL_mass_pi0_samples_names
    name_ends = "_pi0"
if Params["Load_lepton_dirac"] == True: 
    HNL_masses = Constants.HNL_ee_dirac_mass_samples
    sample_names = Constants.HNL_ee_dirac_names
    name_ends = "_ee_dirac"
if Params["Load_pi0_dirac"] == True: 
    HNL_masses = Constants.HNL_pi0_dirac_mass_samples
    sample_names = Constants.HNL_pi0_dirac_names
    name_ends = "_pi0_dirac"
elif Params["Load_single_file"] == True: HNL_masses = [Params["single_file"]]

max_scores, min_scores = [], []
for HNL_mass in HNL_masses:
    max_scores.append(max(Functions.logit(sample_test_dict[str(HNL_mass)+name_ends][f"BDT_output_{HNL_mass}MeV"])))
    min_scores.append(min(Functions.logit(sample_test_dict["overlay"][f"BDT_output_{HNL_mass}MeV"])))
    
max_all_scores = max(max_scores)
min_all_scores = min(min_scores)
print("Maximum score for all signal samples is " + str(max_all_scores))
print("Minimum score for all signal samples is " + str(min_all_scores))                     

max_score_int = np.ceil(max_all_scores)
min_score_int = np.floor(min_all_scores)
print("Maximum integer for bins is " + str(max_score_int))
print("Minimum integer for bins is " + str(min_score_int))

## Looking at correlations of BDT scores

In [ ]:
# sample_test_dict['beamgood']

ee_output_list = ["BDT_output_10MeV", "BDT_output_20MeV", "BDT_output_50MeV", "BDT_output_100MeV", "BDT_output_150MeV"]
pi0_output_list = ["BDT_output_150MeV", "BDT_output_180MeV", "BDT_output_200MeV", "BDT_output_220MeV", "BDT_output_240MeV", "BDT_output_245MeV"]

if Params["Load_lepton_signal"]==True: score_list = ee_output_list
if Params["Load_pi0_signal"]==True: score_list = pi0_output_list

#select only top events, e.g score >5 in one model. 

In [ ]:

method = 'kendall'
correlations = sample_test_dict['beamgood'][score_list].astype(np.float64).corr(method=method)
plt.figure(figsize=(15,12))
sns.heatmap(correlations,vmin=-1,annot=False,square=True,cbar_kws={'label':method+' correlation'},cmap = 'RdBu_r')
plt.title('Input Variable Correlations')
plt.show()


In [ ]:
correlations

## Testing Maj vs Dirac scores

In [ ]:
#Checking which bins to merge Need to properly write this
merged_bins_dict, bin_cents = {}, {}
# min_score_cut = -5.0
min_score_cut = min_score_int
xlims = [min_score_cut,max_score_int]
nbins = int(xlims[1]-xlims[0])
min_overlay = 3 #The threshold of minimum overlay events required in a bin, used to be 1
print("Initial nbins is " + str(nbins))
print("With range of " + str(xlims))
print(f"Requiring more than {min_overlay} overlay events in each bin")

for HNL_mass in HNL_masses:
    bins_list = np.histogram(sample_test_dict['overlay'][f'BDT_output_{HNL_mass}MeV'],bins=nbins,range=xlims)[1]
    if Params["Use_logit"] == True:
        bkg_scores=[Functions.logit(sample_test_dict['overlay'][f'BDT_output_{HNL_mass}MeV']),
                    Functions.logit(sample_test_dict['dirtoverlay'][f'BDT_output_{HNL_mass}MeV']),
                    Functions.logit(sample_test_dict['beamoff'][f'BDT_output_{HNL_mass}MeV'])]
    totbkg=np.histogram(bkg_scores[0],bins=nbins,range=xlims)[0]+np.histogram(bkg_scores[1],bins=nbins,range=xlims)[0]+np.histogram(bkg_scores[2],bins=nbins,range=xlims)[0]
    offbkg=np.histogram(bkg_scores[2],bins=nbins,range=xlims)[0]
    overlaybkg=np.histogram(bkg_scores[0],bins=nbins,range=xlims)[0]
    dirtbkg=np.histogram(bkg_scores[1],bins=nbins,range=xlims)[0]
    bins_new=[]
    for i,bin_bkg in enumerate(totbkg):
        if(overlaybkg[i]>min_overlay): #Checking if unweighted overlay bkg has at least one event in the bin
            bins_new.append(bins_list[i])

    bins_new.append(bins_list[-1])
    merged_bins_dict[HNL_mass] = bins_new
    bin_cents[HNL_mass] = (np.array(bins_new[:-1])+np.array(bins_new[1:]))/2

def make_overflow_bin(bins_dict, bins_cents_dict):
    """
    For making the final "overflow" bin the same size as the previous bins, i.e one integer in width.
    """
    bins_overflow, bins_cent_overflow = {}, {}
    for HNL_mass in bins_dict:
        overflow_bin = bins_cents_dict[HNL_mass][-2]+1 #Just adding one to the penultimate bin centre val. 
        bins_cent_overflow[HNL_mass] = bins_cents_dict[HNL_mass].copy()
        bins_cent_overflow[HNL_mass][-1] = overflow_bin
        bins_overflow[HNL_mass] = bins_dict[HNL_mass].copy()
        bins_overflow[HNL_mass][-1] = bins_dict[HNL_mass][-2]+1 #Just adding one to the penultimate bin end val. 
    return bins_overflow, bins_cent_overflow

bins_overflow, bins_cents_overflow = make_overflow_bin(merged_bins_dict, bin_cents)

def make_xlims_dict(bins_dict, spacing, lower = None):
    """
    Making a dict of xlims for plotting several mass points at once.
    Also returns a dict of xticks for the purpose of indicating the overflow.
    """
    xlims_adjusted, xticks_adjusted = {}, {}
    vals_dict={}
    for HNL_mass in bins_dict:
        if isinstance(lower,(int, float)): lower_val = lower
        else: lower_val = bins_dict[HNL_mass][0]
        xlims_adjusted[HNL_mass] = [lower_val,bins_dict[HNL_mass][-1]]
        ticks = np.arange(bins_dict[HNL_mass][0], bins_dict[HNL_mass][-1], spacing)
        if ticks[-1] != bins_dict[HNL_mass][-2]: ticks = np.append(ticks, bins_dict[HNL_mass][-1]-1)
        ticks_strings = []
        vals = []
        for val in ticks:
            ticks_strings.append(str(int(val)))
            vals.append(val)
        ticks_strings[-1] = str(ticks_strings[-1])+"+"
        xticks_adjusted[HNL_mass] = ticks_strings
        vals_dict[HNL_mass] = vals
        
    return xlims_adjusted, xticks_adjusted, vals_dict

xlims_dict, xticks_dict, vals_dict = make_xlims_dict(bins_overflow, 2)

xlims=[min_score_int,max_score_int]    
BINS = int(xlims[1]-xlims[0])


In [ ]:
HNL_mass = 150
adjacent_mass = 245
score_test = Functions.logit(sample_test_dict[str(HNL_mass)+name_ends][f"BDT_output_{HNL_mass}MeV"])
score_test_adjacent = Functions.logit(sample_test_dict[str(HNL_mass)+name_ends][f"BDT_output_{adjacent_mass}MeV"])

# bins_test = np.arange(min_score_int, max_score_int)
bins_test = merged_bins_dict[HNL_mass]

dir_hist = np.histogram(score_test, bins=bins_test)[0]
# print(len(dir_hist))
# print(len(bins_cents_overflow))

plt.hist(bins_cents_overflow[HNL_mass], bins=bins_overflow[HNL_mass], weights = dir_hist, histtype="step", lw=2, label=f"{HNL_mass} Dirac {HNL_mass} model")

# plt.hist(score_test, bins=bins_test, histtype="step", lw=2, label=f"{HNL_mass} Dirac {HNL_mass} model")
# plt.hist(score_test_adjacent, bins=bins_test, histtype="step", lw=2, label=f"{HNL_mass} Dirac {adjacent_mass} model")

plt.legend()

In [ ]:
Maj_Params = {"Run":"run1", #The run number, so far either "run1" or "run3"
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":245,
          "Load_standard":False, #bkgs
          "Load_lepton_signal":False,
          "Load_lepton_dirac":False,
          "Load_pi0_signal":True,
          "Load_pi0_dirac":False,
          "Load_DetVars":False, #This is for overlay
          "Only_keep_common_DetVar_evs":True,
          "Load_Signal_DetVars":False, #Don't do here, but in seperate script
          'Load_pi0_signal_DetVars':False, #Don't do here, but in seperate script
          "Load_data":False,
          "FLATTEN":True, #Have one row per reconstructed object in the analysis dataframe
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "EXT_in_training":True,
          "dirt_in_training":True,
          "Use_logit":True,
          "nbins":5} 

Maj_samples = Functions.create_test_samples_list(Maj_Params)

Majorana_test_dict = Load_preselected_pkls(Maj_samples, Maj_Params, loc_pkls, "_full_Finished", end_string)

Majorana_test_dict.keys()

# Majorana_test_dict = Load_BDT_models(Majorana_test_dict, Maj_Params, end_string)
Majorana_test_dict = OLD_Load_BDT_models(Majorana_test_dict, Maj_Params, "_Finished")


In [ ]:

Maj_ends = "_pi0"
score_test = Functions.logit(Majorana_test_dict[str(HNL_mass)+Maj_ends][f"BDT_output_{HNL_mass}MeV"])
score_test_adjacent = Functions.logit(Majorana_test_dict[str(HNL_mass)+Maj_ends][f"BDT_output_{adjacent_mass}MeV"])

bins_test = np.arange(min_score_int, max_score_int)

plt.hist(score_test, bins=bins_test, histtype="step", lw=2, label=f"{HNL_mass} Majorana {HNL_mass} model")
plt.hist(score_test_adjacent, bins=bins_test, histtype="step", lw=2, label=f"{HNL_mass} Majorana {adjacent_mass} model")

plt.legend()

In [ ]:
BDT_name = end_string

path_training_fracs = loc+f"Training_fractions/{BDT_name}_"+Params["Run"]

if(os.path.exists(path_training_fracs)):
    with open(path_training_fracs, "rb") as fp:   # Unpickling
        saved_train_fracs = pickle.load(fp)
print(saved_train_fracs)

SF_Maj = 1/(1-saved_train_fracs['signal'])

HNL_mass = 150

POT_ratio = Constants.run1_POT_scaling_dict[f"{HNL_mass}_pi0"]/Constants.run1_POT_scaling_dict[f"{HNL_mass}_pi0_dirac"]
print(POT_ratio)
Maj_scaling = POT_ratio/2.0

Maj_weights = np.ones(len(Majorana_test_dict[str(HNL_mass)+Maj_ends][f"BDT_output_{HNL_mass}MeV"]))
Maj_weights = Maj_weights*SF_Maj*Maj_scaling

score_test_Dir = Functions.logit(sample_test_dict[str(HNL_mass)+name_ends][f"BDT_output_{HNL_mass}MeV"])
score_test_adjacent_Dir = Functions.logit(sample_test_dict[str(HNL_mass)+name_ends][f"BDT_output_{adjacent_mass}MeV"])

score_test_Maj = Functions.logit(Majorana_test_dict[str(HNL_mass)+Maj_ends][f"BDT_output_{HNL_mass}MeV"])
score_test_adjacent_Maj = Functions.logit(Majorana_test_dict[str(HNL_mass)+Maj_ends][f"BDT_output_{adjacent_mass}MeV"])

maj_hist = np.histogram(score_test_Maj, bins=merged_bins_dict[HNL_mass], weights=Maj_weights)[0]

print(merged_bins_dict[HNL_mass])
print(dir_hist)

plt.figure(figsize=[10,8])

plt.hist(bins_cents_overflow[HNL_mass], bins=bins_overflow[HNL_mass], weights = dir_hist, histtype="step", lw=2, label=f"{HNL_mass} Dirac {HNL_mass} model")
plt.hist(bins_cents_overflow[HNL_mass], bins=bins_overflow[HNL_mass], weights = maj_hist, histtype="step", lw=2, label=f"{HNL_mass} Majorana {HNL_mass} model")

# bins_test = np.arange(min_score_int, max_score_int)

# plt.hist(score_test_Dir, bins=bins_test, histtype="step", lw=2, label=f"{HNL_mass} Dirac {HNL_mass} model")
# plt.hist(score_test_Maj, bins=bins_test, weights = Maj_weights, histtype="step", lw=2, label=f"{HNL_mass} Majorana {HNL_mass} model")

plt.legend()

## Merging bins with low stat bkg prediction

In [ ]:
#Checking which bins to merge Need to properly write this
merged_bins_dict, bin_cents = {}, {}
# min_score_cut = -5.0
min_score_cut = min_score_int
xlims = [min_score_cut,max_score_int]
nbins = int(xlims[1]-xlims[0])
min_overlay = 3 #The threshold of minimum overlay events required in a bin, used to be 1
print("Initial nbins is " + str(nbins))
print("With range of " + str(xlims))
print(f"Requiring more than {min_overlay} overlay events in each bin")

for HNL_mass in HNL_masses:
    bins_list = np.histogram(sample_test_dict['overlay'][f'BDT_output_{HNL_mass}MeV'],bins=nbins,range=xlims)[1]
    if Params["Use_logit"] == True:
        bkg_scores=[Functions.logit(sample_test_dict['overlay'][f'BDT_output_{HNL_mass}MeV']),
                    Functions.logit(sample_test_dict['dirtoverlay'][f'BDT_output_{HNL_mass}MeV']),
                    Functions.logit(sample_test_dict['beamoff'][f'BDT_output_{HNL_mass}MeV'])]
    totbkg=np.histogram(bkg_scores[0],bins=nbins,range=xlims)[0]+np.histogram(bkg_scores[1],bins=nbins,range=xlims)[0]+np.histogram(bkg_scores[2],bins=nbins,range=xlims)[0]
    offbkg=np.histogram(bkg_scores[2],bins=nbins,range=xlims)[0]
    overlaybkg=np.histogram(bkg_scores[0],bins=nbins,range=xlims)[0]
    dirtbkg=np.histogram(bkg_scores[1],bins=nbins,range=xlims)[0]
    bins_new=[]
    for i,bin_bkg in enumerate(totbkg):
        if(overlaybkg[i]>min_overlay): #Checking if unweighted overlay bkg has at least one event in the bin
            bins_new.append(bins_list[i])

    bins_new.append(bins_list[-1])
    merged_bins_dict[HNL_mass] = bins_new
    bin_cents[HNL_mass] = (np.array(bins_new[:-1])+np.array(bins_new[1:]))/2
    
    

## Plotting BDT scores (no data)

In [ ]:
#Need to add dirac samples into plot dict here

In [ ]:
BDT_name = end_string

path_training_fracs = loc+f"Training_fractions/{BDT_name}_"+Params["Run"]

if(os.path.exists(path_training_fracs)):
    with open(path_training_fracs, "rb") as fp:   # Unpickling
        saved_train_fracs = pickle.load(fp)
else:
    print("No training fractions saved for this BDT model.\nUsing defaults of overlay=0.7, signal=0.7, EXT=0.3 and dirt=0.0")
    saved_train_fracs = {"overlay":0.7, "signal":0.7, "beamoff":0.3, "dirtoverlay":0.0}
    
print(saved_train_fracs)
    
sample_norms = Functions.Get_weighted_sample_norms(Params, sample_test_dict, sample_names, saved_train_fracs["signal"], saved_train_fracs["overlay"], 
                                                   saved_train_fracs["beamoff"], saved_train_fracs["dirtoverlay"])

if (Params["Load_lepton_dirac"] == True) or (Params["Load_pi0_dirac"]):
    for sample in sample_names:
        signal_scale_list = np.ones(len(sample_test_dict[sample]['n_pfps']))*1.0 #No test sample for Dirac, so no scale factor needed
        sample_norms[sample]=signal_scale_list

samples_plot={'overlay_test':sample_test_dict['overlay'],
              'dirtoverlay':sample_test_dict['dirtoverlay'],
              'beamoff':sample_test_dict['beamoff']}

#Update with signal dfs too.
for signal_name in sample_names:
    samples_plot.update({signal_name:sample_test_dict[signal_name]})
    
print(samples_plot.keys())
print(sample_norms)


In [ ]:
def make_overflow_bin(bins_dict, bins_cents_dict):
    """
    For making the final "overflow" bin the same size as the previous bins, i.e one integer in width.
    """
    bins_overflow, bins_cent_overflow = {}, {}
    for HNL_mass in bins_dict:
        overflow_bin = bins_cents_dict[HNL_mass][-2]+1 #Just adding one to the penultimate bin centre val. 
        bins_cent_overflow[HNL_mass] = bins_cents_dict[HNL_mass].copy()
        bins_cent_overflow[HNL_mass][-1] = overflow_bin
        bins_overflow[HNL_mass] = bins_dict[HNL_mass].copy()
        bins_overflow[HNL_mass][-1] = bins_dict[HNL_mass][-2]+1 #Just adding one to the penultimate bin end val. 
    return bins_overflow, bins_cent_overflow

bins_overflow, bins_cents_overflow = make_overflow_bin(merged_bins_dict, bin_cents)

def make_xlims_dict(bins_dict, spacing, lower = None):
    """
    Making a dict of xlims for plotting several mass points at once.
    Also returns a dict of xticks for the purpose of indicating the overflow.
    """
    xlims_adjusted, xticks_adjusted = {}, {}
    vals_dict={}
    for HNL_mass in bins_dict:
        if isinstance(lower,(int, float)): lower_val = lower
        else: lower_val = bins_dict[HNL_mass][0]
        xlims_adjusted[HNL_mass] = [lower_val,bins_dict[HNL_mass][-1]]
        ticks = np.arange(bins_dict[HNL_mass][0], bins_dict[HNL_mass][-1], spacing)
        if ticks[-1] != bins_dict[HNL_mass][-2]: ticks = np.append(ticks, bins_dict[HNL_mass][-1]-1)
        ticks_strings = []
        vals = []
        for val in ticks:
            ticks_strings.append(str(int(val)))
            vals.append(val)
        ticks_strings[-1] = str(ticks_strings[-1])+"+"
        xticks_adjusted[HNL_mass] = ticks_strings
        vals_dict[HNL_mass] = vals
        
    return xlims_adjusted, xticks_adjusted, vals_dict

xlims_dict, xticks_dict, vals_dict = make_xlims_dict(bins_overflow, 2)

xlims=[min_score_int,max_score_int]    
BINS = int(xlims[1]-xlims[0])


In [ ]:
HNL_scale = 0.1

for sig_name in sample_names:
    HNL_mass = int(sig_name.split("_")[0])
    theta_original = Constants.theta_mu_4_dict[HNL_mass]
    theta_squared = theta_original**2
    POT_scaling = Constants.run1_POT_scaling_dict[sig_name]
    plotted_scaling = HNL_scale
    
    theta_scaling_4 = POT_scaling/plotted_scaling 
    theta_scaling_2 = np.sqrt(theta_scaling_4)
    
    approx_theta_squared = theta_squared/theta_scaling_2
    
    print(sig_name)
    print(approx_theta_squared)
    
xlims_dict = {}
for HNL_mass in bins_overflow:
    xlims_dict[HNL_mass] = [bins_overflow[HNL_mass][0]-1,bins_overflow[HNL_mass][-1]+1]
    

In [ ]:
reload(PT)   
# HNL_scale = 0.1

PT.Plot_BDT_output(HNL_masses=HNL_masses, signal_names=sample_names, samples=samples_plot, sample_norms=sample_norms, colours={}, xlims=xlims_dict,
                   bins_cent_dict=bins_cents_overflow, bins_dict=bins_overflow, bins_original=merged_bins_dict,
                   xticks=xticks_dict, xticks_vals=vals_dict,figsize=[12,8], MergeBins=True, density=False, legloc="best",logy=False, savefig=False, 
                   save_str = "_full_Finished_10", Run=Params["Run"], logit = Params["Use_logit"],legsize=20, HNL_scale=HNL_scale)



## Saving BDT output to .root files

In [ ]:
def Save_one_hist(hist,name,nbins,xlims):
    tData = ROOT.TH1F(name,name,len(nbins)-1,array("d",nbins))
    for i in range(len(nbins)-1):
        tData.SetBinContent(i+1,hist['hist'][i])
        tData.SetBinError(i+1,hist['err'][i])
    return tData
    
def SaveToRoot_new(nbins,xlims,hist_samples,theta,fileName='test.root'): 
    rFile = ROOT.TFile(f'{fileName}','RECREATE')
    tData = ROOT.TH1F("theta","theta",1,array("d",[0,1]))
    tData.SetBinContent(1,theta)
    rFile.Write()
    for i, name in enumerate(hist_samples):
        tData = Save_one_hist(hist_samples[name],name,nbins,xlims)
        rFile.Write()
    rFile.Close()

def make_stat_err_true(var, bins, weights_times_SF): #Saved in Functions.py
    hist_unweighted = np.histogram(var,bins=bins)[0]
    hist_weighted = np.histogram(var,bins=bins,weights=weights_times_SF)[0]
    Total_SF = np.nan_to_num(hist_weighted/hist_unweighted)
    stat_err = np.sqrt(hist_unweighted)*Total_SF
    return stat_err

def make_stat_err_new(var, bins, weights_times_SF): 
    hist_squared = np.histogram(var,bins=bins,weights=weights_times_SF**2)[0]
    stat_err=np.sqrt(hist_squared)
    return stat_err

def SF_times_weights_sample_names(sample_dict, sample, HNL_mass, Run, SF_test, SF_signal):
    if Run == "run1": POT_norm = Constants.run1_POT_scaling_dict
    if Run == "run3": POT_norm = Constants.run3_POT_scaling_dict
    
    if (sample == 'overlay') or (sample == 'dirtoverlay'): SF = sample_dict[sample]['weight']*POT_norm[sample]*SF_test
    elif sample == 'beamoff': SF = np.ones(len(sample_dict[sample][f'run']))*POT_norm[sample]*SF_test
    elif sample =='beamgood': SF = np.ones(len(sample_dict[sample][f'run']))*POT_norm[sample]
    else: 
        SF = np.ones(len(sample_dict[sample][f'BDT_output_{HNL_mass}MeV']))*(POT_norm[sample]*SF_test*SF_signal)
    return SF

## Saving BDT output for reweighting systematics plots

In [ ]:
pkl_results_loc = "pkl_files/"+Params["Run"]+"/current_files/Results/"
if Params["Load_pi0_signal"] == True: pkl_results_loc += "pi0/"

save_name = end_string

print_vals = input("Do you want to save the new BDT scores to .pkl files? y/n ")
if print_vals == "y":
    
    columns = Variables.event_vars + Variables.weight_related + ["weight"]
    for HNL_mass in merged_bins_dict:
        columns.append(f'BDT_output_{HNL_mass}MeV')
    print(columns)
    
    overlay_to_save = sample_test_dict['overlay'][columns].copy()
    overlay_to_save.to_pickle(pkl_results_loc + f"overlay_results{save_name}.pkl")
    
    print(overlay_to_save.keys())
    
    print(pkl_results_loc + f"overlay_results{save_name}.pkl")

## Saving to .root files

In [ ]:
All_SF_train = {}
for sample in samples:
    if sample in saved_train_fracs.keys(): All_SF_train[sample] = saved_train_fracs[sample]
    elif sample in sample_names: All_SF_train[sample] = saved_train_fracs["signal"]
    else: All_SF_train[sample] = 0.0
    
if (Params["Load_lepton_dirac"] == True) or (Params["Load_pi0_dirac"]):
    for sample in sample_names:
        All_SF_train[sample] = 0.0
    
print(All_SF_train)


In [ ]:
# SCALE_UP_dict = {2:5,10:5,20:200,50:50,100:10,150:5,180:4,200:4,220:4,240:4,245:4} #Doing this because the scale factor used in pyhf is bounded
SCALE_UP_dict = {2:1.0,10:1.0,20:1.0,50:1.0,100:1.0,150:1.0, 180:1.0, 200:1.0, 220:1.0, 240:1.0, 245:1.0}

new_theta_dict, dict_for_root = {}, {}

bkg_samples = []
for sample in samples:
    if sample in sample_names: continue
    else: bkg_samples.append(sample)

name_type = Functions.Get_signal_name_type(Params)

print("Saving minimum BDT score of " + str(merged_bins_dict[150][0]))

for i, HNL_mass in enumerate(HNL_masses):
    print(HNL_mass)
    sig_name = sample_names[i]
    original_theta = Constants.theta_mu_4_dict[HNL_mass]
    new_theta = original_theta*SCALE_UP_dict[HNL_mass]
    new_theta_dict[HNL_mass] = original_theta*SCALE_UP_dict[HNL_mass]
    SCALE_EVENTS = SCALE_UP_dict[HNL_mass]**4 #The Number of events is proportional to theta^4
    
    bins = merged_bins_dict[HNL_mass]
    sample_list = bkg_samples + [sig_name]
    
    for sample in sample_list:
        train_vs_test_fraction = All_SF_train[sample]
        SF_test = 1.0/(1-train_vs_test_fraction)
        
        score = Functions.logit(sample_test_dict[sample][f'BDT_output_{HNL_mass}MeV'])
        SF_list = SF_times_weights_sample_names(sample_test_dict, sample, HNL_mass, Params["Run"], SF_test, SCALE_EVENTS)
        stat_err = make_stat_err_true(score, bins, SF_list) #Should generally use this
        # stat_err = make_stat_err_new(score, bins, SF_list) #Can use if I get a Nan or inf error
        hist = np.histogram(score, bins=bins, weights=SF_list)
        dict_for_root[sample] = {'bins': np.array(hist[1]), 'hist': np.array(hist[0]), 'err': np.array(stat_err)}
        
    hist_samples = {"bkg_overlay":dict_for_root['overlay'],"bkg_dirt":dict_for_root['dirtoverlay'],
                    "bkg_EXT":dict_for_root['beamoff'],"signal":dict_for_root[sample]}
    if 'beamgood' in bkg_samples:
        hist_samples.update({"data":dict_for_root['beamgood']})
        
    save_name = Params["Run"]+f"_{HNL_mass}_{name_type}{end_string}.root"
    
    if (Params["Load_pi0_signal"]==True) or (Params["Load_pi0_dirac"]==True):
        SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="bdt_output/pi0/"+save_name)
        SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="Uncertainties/pi0/"+save_name)
    else:
        SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="bdt_output/"+save_name)
        SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="Uncertainties/"+save_name)
        
print("Saved all")
print(f"Saved with names like {save_name}")


## Calculating "efficiency" of BDT separation

In [ ]:
def calculate_efficiency(df, file_name, Run, BDT_cutoff, HNL_mass, train_fracs, debug=False):
    if Run == "run1": ev_num_dict = Constants.run1_event_numbers
    if Run == "run3": ev_num_dict = Constants.run3_event_numbers
    
    train_vs_test_fraction = train_fracs[file_name] #Should be saved in in script 3.0
    norm = 1.0/(1-train_vs_test_fraction) #Need to scale up to account for events lost
    
    total_ev_num = len(df)
    ev_num_initial = ev_num_dict[file_name]
    cut_df = df.query(f"BDT_output_{HNL_mass}MeV > {BDT_cutoff}")
    new_ev_num = len(cut_df)
    
    BDT_efficiency = new_ev_num/total_ev_num
    Total_efficiency = norm*(new_ev_num/ev_num_initial)
    
    if debug:
        print(f"Total events is {total_ev_num}")
        print(f"Events after cut is {new_ev_num}")   
    
    return BDT_efficiency, Total_efficiency

def calculate_ev_numbers(df, file_name, Run, BDT_cutoff, HNL_mass, train_fracs, debug=False):
    if Run == "run1": 
        ev_num_dict = Constants.run1_event_numbers
        POT_scaling_dict = Constants.run1_POT_scaling_dict
    if Run == "run3": 
        ev_num_dict = Constants.run3_event_numbers
        POT_scaling_dict = Constants.run3_POT_scaling_dict
        
    train_vs_test_fraction = train_fracs[file_name] #Should be saved in in script 3.0
    norm = 1.0/(1-train_vs_test_fraction) #Need to scale up to account for events lost
        
    cut_df = df.query(f"BDT_output_{HNL_mass}MeV > {BDT_cutoff}")
    
    if file_name == "overlay" or file_name == "dirtoverlay":
        sum_weights = cut_df["weight"].sum()
    else: sum_weights = len(cut_df)
    
    Number_events = sum_weights*POT_scaling_dict[file_name]*norm
    
    return Number_events
    

def sum_total_weights(df): #This is to check that the weighting doesn't affect the overall efficiency significantly (i.e av weight is approx 1)
    total_evs = len(df)
    sum_weights = df["weight"].sum()
    
    weight_per_ev = sum_weights/total_evs
    
    print(f"Sum of weights is {sum_weights}")
    print(f"Average weight is {weight_per_ev}")


In [ ]:
print(sample_test_dict.keys())
# BDT_name = end_string
BDT_name = "_full_Finished_10"

if Params["Load_lepton_signal"]==True: loc = "bdts/"
if Params["Load_pi0_signal"]==True: loc = "bdts/pi0_selection/"

with open(loc+f"Training_fractions/{BDT_name}_"+Params["Run"], "rb") as fp:   # Unpickling
    saved_train_fracs = pickle.load(fp)
    
# for sample in ['overlay', 'dirtoverlay', 'beamoff']: #sample_test_dict
for sample in sample_test_dict:
    if sample in Constants.HNL_ee_samples_names + Constants.HNL_mass_pi0_samples_names: 
        saved_train_fracs[sample] = saved_train_fracs["signal"]
    elif sample not in saved_train_fracs.keys(): saved_train_fracs[sample] = 0.0

print(saved_train_fracs)

In [ ]:
Number_of_bins = 3

BDT_cut = {}

for sample in sample_names:
    HNL_mass = int(sample.split("_")[0])
    BDT_cut[sample] = Functions.invlogit(merged_bins_dict[HNL_mass][-1*Number_of_bins])
    
if name_type == "ee": standard_BDT_cut = BDT_cut["100_ee"]
if name_type == "pi0": standard_BDT_cut = BDT_cut["200_pi0"]
print(BDT_cut)
print(standard_BDT_cut)

In [ ]:

logit_BDT_cut = Functions.logit(standard_BDT_cut)
print(f"Using a logit BDT cut of {logit_BDT_cut}, which corresponds to {standard_BDT_cut} in [0,1] BDT score.")

BDT_efficiency_dict = {}
Total_efficiency_dict = {}
Scaled_events_dict = {}

for file in sample_names: #Just signal samples
# for file in sample_test_dict:
    HNL_mass_number = file.split("_")[0]
    print(file)
    BDT_efficiency_dict[file], Total_efficiency_dict[file] = calculate_efficiency(sample_test_dict[file], file, Params["Run"], BDT_cut[file], HNL_mass_number,
                                                 saved_train_fracs, debug=True)
    Scaled_events_dict[file] = calculate_ev_numbers(sample_test_dict[file], file, Params["Run"], BDT_cut[file], HNL_mass_number,
                                                 saved_train_fracs, debug=True)


In [ ]:
if name_type == "ee": bkg_mass_model = "100_ee"
if name_type == "pi0": bkg_mass_model = "200_pi0"

for file in ['overlay', 'dirtoverlay', 'beamoff']: #Calculating efficiency for 100MeV BDT model score
    print(file)
    bkg_mass = int(bkg_mass_model.split("_")[0])
    BDT_efficiency_dict[file], Total_efficiency_dict[file] = calculate_efficiency(sample_test_dict[file], file, Params["Run"], standard_BDT_cut, bkg_mass, 
                                                 saved_train_fracs, debug=True)
    Scaled_events_dict[file] = calculate_ev_numbers(sample_test_dict[file], file, Params["Run"], standard_BDT_cut, bkg_mass,
                                                 saved_train_fracs, debug=True)
    
Total_predicted_presel = calculate_ev_numbers(sample_test_dict["overlay"], "overlay", Params["Run"], 0.0, HNL_mass, saved_train_fracs)
Total_predicted_presel += calculate_ev_numbers(sample_test_dict["dirtoverlay"], "dirtoverlay", Params["Run"], 0.0, HNL_mass, saved_train_fracs)
Total_predicted_presel += calculate_ev_numbers(sample_test_dict["beamoff"], "beamoff", Params["Run"], 0.0, HNL_mass, saved_train_fracs)

print(f"Total presel events is {Total_predicted_presel}")


In [ ]:
print(Params["Run"] + " efficiencies are:")
for file in sample_test_dict:
    if file=="beamgood": continue
    # print(f"{file} efficiency is " + str(efficiency_dict[file]))
    print(f"{file} BDT efficiency is " + str(BDT_efficiency_dict[file]*100) + " %")
    print(f"{file} Total efficiency is " + str(Total_efficiency_dict[file]*100) + " %")
    
    print(f"{file} scaled number of events is " + str(Scaled_events_dict[file]))
    
predicted_sum = Scaled_events_dict["overlay"] + Scaled_events_dict["dirtoverlay"] + Scaled_events_dict["beamoff"]
print("----------------------------------")
print(f"Total predicted bkg events is {predicted_sum}")

In [ ]:

Total_ev_numbers = {}
if Params["Run"] == "run1":
    Presel_effic = {"beamoff":0.007, "overlay":0.049, "dirtoverlay":0.012, "combined":0.012}
    Ev_nums_dict = Constants.run1_event_numbers
if Params["Run"] == "run3":
    Presel_effic = {"beamoff":0.004, "overlay":0.038, "dirtoverlay":0.008, "combined":0.009}
    Ev_nums_dict = Constants.run3_event_numbers

for file in ["beamoff", "overlay", "dirtoverlay"]:
    
    total_effic = BDT_efficiency_dict[file]*Presel_effic[file]
    print(file + ": " + str(total_effic))
    
print("------------------------")
sum_predictions_eff = predicted_sum/Total_predicted_presel
print(f"sum prediction BDT eff: {sum_predictions_eff*100}")
sum_predictions_total_eff = sum_predictions_eff*Presel_effic["combined"]
print(f"sum prediction Total eff: {sum_predictions_total_eff*100}")

## "Closure test" i.e testing adjacent model output

In [ ]:
sample = "10_ee"
sample_mass = int(sample.split("_")[0])
model = 20
SCALE_UP_dict = {2:1.0,10:1.0,20:1.0,50:1.0,100:1.0,150:1.0, 180:1.0, 200:1.0, 220:1.0, 240:1.0, 245:1.0}

train_vs_test_fraction = All_SF_train[sample]
SF_test = 1.0/(1-train_vs_test_fraction)

# bins = merged_bins_dict[sample_mass] #Should these bins be used?
bins = merged_bins_dict[model]

score = Functions.logit(sample_test_dict[sample][f'BDT_output_{model}MeV'])
SF_list = SF_times_weights_sample_names(sample_test_dict, sample, sample_mass, Params["Run"], SF_test, SCALE_UP_dict[sample_mass])
stat_err = make_stat_err_true(score, bins, SF_list) #Should generatlly use this
# stat_err = make_stat_err_new(score, bins, SF_list)
hist, bin_edges = np.histogram(score, bins=bins, weights=SF_list)

print(hist)
print(bin_edges)
print(bins)


# cents = bin_cents[sample_mass]
cents = bin_cents[model]

plot = plt.hist(cents, bins=bins, weights=hist, histtype="step")
plt.show()

In [ ]:
sample = "10_ee"
sample_mass = int(sample.split("_")[0])
model = 20

# sample_test_dict[sample][f'BDT_output_{model}MeV']
bins = merged_bins_dict[sample_mass]

plt.hist(Functions.logit(sample_test_dict[sample][f'BDT_output_{model}MeV']), bins=bins, histtype="step", label=f"{sample}_{model}")
plt.hist(Functions.logit(sample_test_dict[sample][f'BDT_output_{sample_mass}MeV']), bins=bins, histtype="step", label=f"{sample}_{sample_mass}")

plt.legend()


In [ ]:
sample = "20_ee"
sample_mass = int(sample.split("_")[0])
model = 10

# sample_test_dict[sample][f'BDT_output_{model}MeV']
bins = merged_bins_dict[sample_mass]
bins_model = merged_bins_dict[model]

# plt.hist(Functions.logit(sample_test_dict[sample][f'BDT_output_{model}MeV']), bins=bins, histtype="step", label=f"{sample}_{model}")
plt.hist(Functions.logit(sample_test_dict[sample][f'BDT_output_{model}MeV']), bins=bins_model, histtype="step", label=f"{sample}_{model}")
# plt.hist(Functions.logit(sample_test_dict[sample][f'BDT_output_{sample_mass}MeV']), bins=bins, histtype="step", label=f"{sample}_{sample_mass}")
plt.hist(Functions.logit(sample_test_dict[sample][f'BDT_output_{sample_mass}MeV']), bins=bins, histtype="step", label=f"{sample}_{sample_mass}")

plt.legend()

In [ ]:
new_theta_dict, dict_for_root = {}, {}

SCALE_UP_dict = {2:1.0,10:1.0,20:1.0,50:1.0,100:1.0,150:1.0, 180:1.0, 200:1.0, 220:1.0, 240:1.0, 245:1.0}

bkg_samples = []
for sample in samples:
    if sample in sample_names: continue
    else: bkg_samples.append(sample)

name_type = Functions.Get_signal_name_type(Params)

print("Saving minimum BDT score of " + str(merged_bins_dict[150][0]))

mass_list = list(merged_bins_dict.keys())

for i, test_sig in enumerate(sample_names):
    print(test_sig)
    HNL_mass = int(test_sig.split("_")[0])
    original_theta = Constants.theta_mu_4_dict[HNL_mass]
    new_theta = original_theta*SCALE_UP_dict[HNL_mass]
    new_theta_dict[HNL_mass] = original_theta*SCALE_UP_dict[HNL_mass]
    SCALE_EVENTS = SCALE_UP_dict[HNL_mass]**4 #The Number of events is proportional to theta^4
    
    # bins = merged_bins_dict[HNL_mass] #Should these bins be used?

    sample_list = bkg_samples + [test_sig]

    model_masses = [] #for masses either side of test point (if available)
    
    if i > 0: model_masses.append(mass_list[i-1])
    if i < len(sample_names)-1: model_masses.append(mass_list[i+1])
    
    for model in model_masses: #Looping over adjacent models
        bins = merged_bins_dict[model]
        for sample in sample_list:
            train_vs_test_fraction = All_SF_train[sample]
            SF_test = 1.0/(1-train_vs_test_fraction)

            score = Functions.logit(sample_test_dict[sample][f'BDT_output_{model}MeV'])
            SF_list = SF_times_weights_sample_names(sample_test_dict, sample, HNL_mass, Params["Run"], SF_test, SCALE_EVENTS)
            stat_err = make_stat_err_true(score, bins, SF_list) #Should generally use this
            # stat_err = make_stat_err_new(score, bins, SF_list)
            hist, bin_edges = np.histogram(score, bins=bins, weights=SF_list)
            # dict_for_root[sample] = {'bins': np.array(hist[1]), 'hist': np.array(hist[0]), 'err': np.array(stat_err)}
            dict_for_root[sample] = {'bins': bin_edges, 'hist': hist, 'err': stat_err}

        hist_samples = {"bkg_overlay":dict_for_root['overlay'],"bkg_dirt":dict_for_root['dirtoverlay'],
                        "bkg_EXT":dict_for_root['beamoff'],"signal":dict_for_root[sample]}
        if 'beamgood' in bkg_samples:
            hist_samples.update({"data":dict_for_root['beamgood']})

        save_name = Params["Run"]+f"_corrected_Test_{HNL_mass}_{name_type}_model_{model}{end_string}.root"

        if (Params["Load_pi0_signal"]==True) or (Params["Load_pi0_dirac"]==True):
            # SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="bdt_output/pi0/adjacent_models/"+save_name)
            # SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="Uncertainties/pi0/adjacent_models/"+save_name)
            SaveToRoot_new(merged_bins_dict[model],xlims,hist_samples,theta=new_theta,fileName="bdt_output/pi0/adjacent_models/"+save_name)
            SaveToRoot_new(merged_bins_dict[model],xlims,hist_samples,theta=new_theta,fileName="Uncertainties/pi0/adjacent_models/"+save_name)
        else:
            # SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="bdt_output/adjacent_models/"+save_name)
            # SaveToRoot_new(merged_bins_dict[HNL_mass],xlims,hist_samples,theta=new_theta,fileName="Uncertainties/adjacent_models/"+save_name)
            SaveToRoot_new(merged_bins_dict[model],xlims,hist_samples,theta=new_theta,fileName="bdt_output/adjacent_models/"+save_name)
            SaveToRoot_new(merged_bins_dict[model],xlims,hist_samples,theta=new_theta,fileName="Uncertainties/adjacent_models/"+save_name)
        
print("Saved all")
print(f"Saved with names like {save_name}")

In [ ]:
# mass_point = 50
mass_point = 240

savefig = False

test_models = [180, 245]

test_results, test_labels = [], []

plt.figure(figsize=[12,8],facecolor='white')

for mass_model in test_models:
    test_results.append(Functions.logit(samples_plot[mass_point][f'BDT_output_{mass_model}MeV']))
    test_labels.append(f"{mass_model} MeV model")

plot=plt.hist(Functions.logit(samples_plot[mass_point][f'BDT_output_{mass_point}MeV']),label=f"Correct {mass_point} MeV model",
                         range=xlims,bins=merged_bins_dict[mass_point],histtype="step",
                         stacked=False,density=False,linewidth=4)
plot=plt.hist(test_results,label=test_labels,range=xlims,bins=merged_bins_dict[mass_point],histtype="step",
              stacked=False,density=False,linewidth=2)

plt.xlabel(f"BDT score for {mass_point} MeV sample")
plt.ylabel("Events")
plt.legend(loc="upper left")
if savefig == True:
    plt.savefig("plots/BDT_output/Adjacent_models_tests/" + Params["Run"] + "_" + str(mass_point) + "MeV_adjcent_models.pdf")
    plt.savefig("plots/BDT_output/Adjacent_models_tests/" + Params["Run"] + "_" + str(mass_point) + "MeV_adjcent_models.png")
plt.show()

## Looking at variables for signal-like events vs. full sample

In [ ]:
cutoff = 0.95
signal_like = {}
frac_retained = {}
pkl_variable_tests_loc = "pkl_files/"+Params["Run"]+"/current_files/Variable_tests/"

Number_signal = 1000
top_signal = {}

# for HNL_mass in Constants.HNL_mass_samples:
for signal_sample in sample_names:
    HNL_mass = int(signal_sample.split("_")[0])
    signal_like[signal_sample] = sample_test_dict[signal_sample].query(f"BDT_output_{HNL_mass}MeV > {cutoff}")
    frac_retained[signal_sample] = len(signal_like[signal_sample])/len(sample_test_dict[signal_sample])
    
    # signal_like[HNL_mass].to_pickle(pkl_variable_tests_loc + f"signal_like_{HNL_mass}MeV.pkl")
    
    top_signal[signal_sample] = sample_test_dict[signal_sample].sort_values(f"BDT_output_{HNL_mass}MeV", ascending=False).head(Number_signal)
    
print(frac_retained)


In [ ]:
plt.figure(figsize=[12,8],facecolor='white')

if name_type == "ee": HNL_mass = "100_ee"
if name_type == "pi0": HNL_mass = "200_pi0"

variable = 'shr_theta_v'
#'shr_theta_v', 'shr_phi_v', 'trk_theta_v', 'trk_phi_v'

bins = 40
xlims = [0,3.2]

plt.hist(sample_test_dict[HNL_mass][variable],
#               label=[f"HNL ({mass} MeV) \n $|U_{{\mu4}}|^2="+sci_notation(sample_info["300"]["theta_u2"]) +f" (x{HNLplotscale})"],
         label=[f"All preselected {HNL_mass} MeV HNL"],
         range=xlims,bins=bins,
         stacked=False,density=True,
         histtype="step",lw=3)

plt.hist(top_signal[HNL_mass][variable],
#               label=[f"HNL ({mass} MeV) \n $|U_{{\mu4}}|^2="+sci_notation(sample_info["300"]["theta_u2"]) +f" (x{HNLplotscale})"],
         label=[f"Signal-like {HNL_mass} MeV HNL"],
         range=xlims,bins=bins,
         stacked=False,density=True,
         histtype="step",lw=3)
plt.xlabel(variable)
plt.legend()

## Make list of run subrun event which are most signal-like

In [ ]:
top_N = 50
csv_location = "bdt_output/signal_like_rse_lists/"
for signal_sample in sample_names:
    HNL_mass = int(signal_sample.split("_")[0])
    signal_sorted = sample_test_dict[signal_sample].sort_values(f"BDT_output_{HNL_mass}MeV", ascending=False).head(top_N)
    overlay_sorted = sample_test_dict['overlay'].sort_values(f"BDT_output_{HNL_mass}MeV", ascending=False).head(top_N)
    dirt_sorted = sample_test_dict['dirtoverlay'].sort_values(f"BDT_output_{HNL_mass}MeV", ascending=False).head(top_N)
    beamoff_sorted = sample_test_dict['beamoff'].sort_values(f"BDT_output_{HNL_mass}MeV", ascending=False).head(top_N)
    beamgood_sorted = sample_test_dict['beamgood'].sort_values(f"BDT_output_{HNL_mass}MeV", ascending=False).head(top_N)
    
    signal_run_sub_event = signal_sorted[["run", "sub", "evt"]]
    overlay_run_sub_event = overlay_sorted[["run", "sub", "evt"]]
    dirt_run_sub_event = dirt_sorted[["run", "sub", "evt"]]
    beamoff_run_sub_event = beamoff_sorted[["run", "sub", "evt"]]
    beamgood_run_sub_event = beamgood_sorted[["run", "sub", "evt"]]
    
    csv_name = csv_location + Params["Run"] + f"_signal_{signal_sample}.csv"
    csv_overlay_name = csv_location + Params["Run"] + f"_overlay_{signal_sample}.csv"
    csv_dirt_name = csv_location + Params["Run"] + f"_dirtoverlay_{signal_sample}.csv"
    csv_beamoff_name = csv_location + Params["Run"] + f"_beamoff_{signal_sample}.csv"
    csv_beamgood_name = csv_location + Params["Run"] + f"_beamgood_{signal_sample}.csv"

    signal_run_sub_event.to_csv(csv_name, sep=" ", header=False, index=False) #i.e separating by a space, removing column names
    overlay_run_sub_event.to_csv(csv_overlay_name, sep=" ", header=False, index=False)
    dirt_run_sub_event.to_csv(csv_dirt_name, sep=" ", header=False, index=False)
    beamoff_run_sub_event.to_csv(csv_beamoff_name, sep=" ", header=False, index=False)
    beamgood_run_sub_event.to_csv(csv_beamgood_name, sep=" ", header=False, index=False)
    
    #signal
    csv_file = csv_name
    txt_file = csv_location + Params["Run"] + f"_signal_{signal_sample}.list" #could save this as .txt, just following Owen's naming system so using .list
    with open(txt_file, "w") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()
        
    #overlay
    csv_file = csv_overlay_name
    txt_file = csv_location + Params["Run"] + f"_overlay_{signal_sample}.list" #could save this as .txt, just following Owen's naming system so using .list
    with open(txt_file, "w") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()
        
    #dirt
    csv_file = csv_dirt_name
    txt_file = csv_location + Params["Run"] + f"_dirtoverlay_{signal_sample}.list" #could save this as .txt, just following Owen's naming system so using .list
    with open(txt_file, "w") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()
        
    #beamoff
    csv_file = csv_beamoff_name
    txt_file = csv_location + Params["Run"] + f"_beamoff_{signal_sample}.list" #could save this as .txt, just following Owen's naming system so using .list
    with open(txt_file, "w") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()
        
    #beamgood
    csv_file = csv_beamgood_name
    txt_file = csv_location + Params["Run"] + f"_beamgood_{signal_sample}.list" #could save this as .txt, just following Owen's naming system so using .list
    with open(txt_file, "w") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()
        
print("Saved all!")

# End of code